In [1]:
import pandas as pd
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from rdkit.Chem import MACCSkeys
import numpy as np
np.set_printoptions(threshold=np.inf)
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

## Concatenating all the files together keeping on the solubility and the SMILES string which will be used to generate rdkit

In [2]:
# Ask Jo about whether to keep the temperature in as the calculated average or remove it
aqSolDB = pd.read_csv("AQSOLDB.csv")[['SMILES','Solubility']].rename(columns={'Solubility' : 'logS' }, inplace=True)
esol = pd.read_csv("ESOL.csv")[['SMILES','measured log(solubility:mol/L)']].rename(columns = {'measured log(solubility:mol/L)': 'logS'}, inplace = True)
novel = pd.read_excel('Improved Prediction of Aqueous Solubility of Novel Compounds by Going Deeper With Deep Learning.xlsx', engine='openpyxl', verbose=False)[['SMILES','logS']]
GCN = pd.read_csv('Multi-channel GCN dataset.csv')[['SMILES','Experimental Solubility in Water']].rename(columns={'Experimental Solubility in Water':'logS'},inplace=True)
OLS = pd.read_excel('OLS_Lasso_High-temperature-water.xlsx', engine='openpyxl', verbose=False)[['SMILES','logS(mol/kg)']].rename(columns={'logS(mol/kg)':'logS'})
datasets = [aqSolDB,esol,GCN,OLS,novel]
data = pd.concat(datasets, ignore_index=True)
data.dropna(inplace=True)
print(data.head())
print(data.shape)

     SMILES  logS
0   CC(N)=O  1.58
1       CNN  1.34
2   CC(=O)O  1.22
3   C1CCNC1  1.15
4  NC(=O)NO  1.12
(11287, 2)


In [5]:
def canonical_smiles(smiles):
    mols = [Chem.MolFromSmiles(smi) for smi in smiles] 
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    return smiles
data["SMILES"] = canonical_smiles(data['SMILES'])
# Create a list for duplicate smiles
duplicates_smiles = data[data['SMILES'].duplicated()]['SMILES'].values
len(duplicates_smiles)
data[data['SMILES'].isin(duplicates_smiles)].sort_values(by=['SMILES'])
dataset_new = data.drop_duplicates(subset=['SMILES'])
len(dataset_new)

10194

In [4]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles] 
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()
    
    Mol_descriptors =[]
    for mol in mols:
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names 

# Function call
Mol_descriptors,desc_names = RDkit_descriptors(dataset_new['SMILES'])
df_with_200_descriptors = pd.DataFrame(Mol_descriptors,columns=desc_names)
df_with_200_descriptors

KeyboardInterrupt: 

In [7]:
df_with_200_descriptors.columns

Index(['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex',
       'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt',
       'NumValenceElectrons', 'NumRadicalElectrons',
       ...
       'fr_sulfide', 'fr_sulfonamd', 'fr_sulfone', 'fr_term_acetylene',
       'fr_tetrazole', 'fr_thiazole', 'fr_thiocyan', 'fr_thiophene',
       'fr_unbrch_alkane', 'fr_urea'],
      dtype='object', length=208)

In [8]:
from rdkit.Chem import PandasTools
data = PandasTools.AddMoleculeColumnToFrame(dataset_new,'SMILES','Molecule')
dataset = pd.merge(dataset_new, df_with_200_descriptors, left_index=True, right_index=True)
print(dataset.head)

<bound method NDFrame.head of                     SMILES      logS  \
0                  CC(N)=O  1.580000   
1                      CNN  1.340000   
2                  CC(=O)O  1.220000   
3                  C1CCNC1  1.150000   
4                 NC(=O)NO  1.120000   
...                    ...       ...   
10189  CC(=O)N1CC(=O)NC1=S -0.234407   
10190     NC(=O)CN1CCCC1=O -0.233746   
10191         Br.CN(C)CCBr -0.233718   
10192        CCN(CC)C(N)=O -0.231606   
10193          CC1(C)NCCS1 -0.225732   

                                                Molecule  MaxEStateIndex  \
0      <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...       10.152778   
1      <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...        6.500000   
2      <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...        9.861111   
3      <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...        7.192708   
4      <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...       10.118194   
...                              

C:\Users\tosin\miniconda3\lib\site-packages\rdkit\Chem\PandasTools.py:379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[molCol] = frame[smilesCol].map(Chem.MolFromSmiles)


In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
print(dataset.columns)

Index(['SMILES', 'logS', 'Molecule', 'MaxEStateIndex', 'MinEStateIndex',
       'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt',
       'HeavyAtomMolWt',
       ...
       'fr_sulfide', 'fr_sulfonamd', 'fr_sulfone', 'fr_term_acetylene',
       'fr_tetrazole', 'fr_thiazole', 'fr_thiocyan', 'fr_thiophene',
       'fr_unbrch_alkane', 'fr_urea'],
      dtype='object', length=211)


In [12]:
dataset['MACCSkeys'] = dataset['Molecule'].apply(lambda x : Chem.AllChem.GetMACCSKeysFingerprint(x))
dataset['adjacenyMatrix'] = dataset['Molecule'].apply(lambda x : Chem.AllChem.GetAdjacencyMatrix(x))
dataset['atom_pair_fingerprint'] = dataset['Molecule'].apply(lambda  x : np.array(Chem.AllChem.GetHashedAtomPairFingerprintAsBitVect(x)))
dataset['morgan_fingerprint'] = dataset['Molecule'].apply(lambda x : np.array(Chem.AllChem.GetMorganFingerprintAsBitVect(x,2,1024)))
dataset['topological_fingerpint'] = dataset['Molecule'].apply(lambda x : np.array(Chem.AllChem.GetHashedTopologicalTorsionFingerprintAsBitVect(x)))
print(dataset.head())

     SMILES  logS                                           Molecule  \
0   CC(N)=O  1.58  <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...   
1       CNN  1.34  <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...   
2   CC(=O)O  1.22  <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...   
3   C1CCNC1  1.15  <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...   
4  NC(=O)NO  1.12  <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...   

   MaxEStateIndex  MinEStateIndex  MaxAbsEStateIndex  MinAbsEStateIndex  \
0       10.152778       -2.857639          10.152778           0.451389   
1        6.500000       -2.718750           6.500000           0.229167   
2        9.861111       -2.789931           9.861111           1.502315   
3        7.192708       -3.078125           7.192708           0.291667   
4       10.118194       -1.356481          10.118194           0.305556   

        qed   MolWt  HeavyAtomMolWt  ...  fr_thiazole  fr_thiocyan  \
0  0.401031  59.068          54.028  ...      

In [13]:
from rdkit.Chem import rdMolDescriptors
dataset['MQNs'] = dataset.Molecule.apply(lambda  x : rdMolDescriptors.MQNs_(x))
print(dataset.head())

     SMILES  logS                                           Molecule  \
0   CC(N)=O  1.58  <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...   
1       CNN  1.34  <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...   
2   CC(=O)O  1.22  <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...   
3   C1CCNC1  1.15  <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...   
4  NC(=O)NO  1.12  <rdkit.Chem.rdchem.Mol object at 0x000002A9D7F...   

   MaxEStateIndex  MinEStateIndex  MaxAbsEStateIndex  MinAbsEStateIndex  \
0       10.152778       -2.857639          10.152778           0.451389   
1        6.500000       -2.718750           6.500000           0.229167   
2        9.861111       -2.789931           9.861111           1.502315   
3        7.192708       -3.078125           7.192708           0.291667   
4       10.118194       -1.356481          10.118194           0.305556   

        qed   MolWt  HeavyAtomMolWt  ...  fr_thiocyan  fr_thiophene  \
0  0.401031  59.068          54.028  ...     

In [15]:
print(type(dataset))
dataset = dataset.dropna()
dataset = dataset.reset_index(drop=True)
dataset.to_csv('dataset.csv',header=True)

<class 'pandas.core.frame.DataFrame'>


In [16]:
print(dataset.shape)

(8879, 217)


In [17]:
print(dataset.iloc[:,5:-1].shape)

(8879, 211)


In [18]:
!conda list

# packages in environment at C:\Users\tosin\miniconda3:
#
# Name                    Version                   Build  Channel
_tflow_select             2.3.0                       mkl  
abseil-cpp                20211102.0           hd77b12b_0  
absl-py                   1.3.0            py39haa95532_0  
aiohttp                   3.8.1            py39h2bbff1b_1  
aiosignal                 1.2.0              pyhd3eb1b0_0  
anyio                     3.5.0            py39haa95532_0  
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0           py39h2bbff1b_0  
asttokens                 2.0.5              pyhd3eb1b0_0  
astunparse                1.6.3                      py_0  
async-timeout             4.0.2            py39haa95532_0  
attrs                     21.4.0             pyhd3eb1b0_0  
babel                     2.9.1              pyhd3eb1b0_0  
backcall                  0.2.0              pyhd3eb1b0_0  
beautifulsoup4            4.11.1   